In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.preprocessing.image import load_img 

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('lords.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='new', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [14]:
fashion_mnist=keras.datasets.fashion_mnist


In [15]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [16]:
#scalling
train_images=train_images/255.0
test_images=test_images/255.0

In [18]:
train_images[0].shape


(28, 28)

In [20]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [21]:
train_images[0].shape

(28, 28, 1)

In [62]:
def build_model(hp):
    model=keras.Sequential([keras.layers.Conv2D(
    filters=hp.Int('convo_1_filter',min_value=32,max_value=128,step=16),
    kernel_size=hp.Choice('conv_1_kerel',values=[3,5]),
    activation='relu',
    input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
    filters=hp.Int('convo_2_filter',min_value=32,max_value=68,step=16),
    kernel_size=hp.Choice('conv_2_kerel',values=[3,5]),
    activation='relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
           units=hp.Int('dense_1',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
#                             last dense layer
        keras.layers.Dense(10,'softmax')
                            
                           ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                 
                 loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

In [63]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [64]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [65]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 11s]
val_accuracy: 0.8793333172798157

Best val_accuracy So Far: 0.9053333401679993
Total elapsed time: 00h 05m 57s
INFO:tensorflow:Oracle triggered exit


In [67]:
model=tuner_search.get_best_models(num_models=1)[0]

In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 48)        1248      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        76864     
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 80)                2048080   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 2,127,002
Trainable params: 2,127,002
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=2)

Epoch 3/10
   5/1688 [..............................] - ETA: 33s - loss: 0.1594 - accuracy: 0.9375 

2022-07-25 17:53:17.159787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1688/1688 [==============================] - ETA: 0s - loss: 0.2125 - accuracy: 0.9210

2022-07-25 17:53:39.541339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1688/1688 [==============================] - 24s 14ms/step - loss: 0.2125 - accuracy: 0.9210 - val_loss: 0.2589 - val_accuracy: 0.9047
Epoch 4/10
1688/1688 [==============================] - 25s 15ms/step - loss: 0.1685 - accuracy: 0.9363 - val_loss: 0.2618 - val_accuracy: 0.9117
Epoch 5/10
1688/1688 [==============================] - 24s 14ms/step - loss: 0.1308 - accuracy: 0.9509 - val_loss: 0.2972 - val_accuracy: 0.9078
Epoch 6/10
1688/1688 [==============================] - 24s 14ms/step - loss: 0.0984 - accuracy: 0.9634 - val_loss: 0.3339 - val_accuracy: 0.9090
Epoch 7/10
1688/1688 [==============================] - 24s 14ms/step - loss: 0.0802 - accuracy: 0.9696 - val_loss: 0.3406 - val_accuracy: 0.9103
Epoch 8/10
1688/1688 [==============================] - 24s 14ms/step - loss: 0.0603 - accuracy: 0.9775 - val_loss: 0.4114 - val_accuracy: 0.9090
Epoch 9/10
1688/1688 [==============================] - 24s 14ms/step - loss: 0.0465 - accuracy: 0.9826 - val_loss: 0.4299 - val_accura